In [25]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M

cfg = GRPO_WRITING_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-04-01 20:19:52.140 | INFO     | trl_wrapper.trainer_wrapper:__init__:285 - Setting padding side to: left
2025-04-01 20:19:52.140 | INFO     | trl_wrapper.trainer_wrapper:init_model:296 - Using device: mps
2025-04-01 20:19:52.141 | INFO     | trl_wrapper.trainer_wrapper:init_model:310 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [27]:
wrapper.init_data_module()

2025-04-01 20:19:52.850 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:336 - Using chat template override: smollmv2
2025-04-01 20:19:52.851 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/writing_g_r_p_o_data_module


In [75]:
import pandas as pd
dataset_pd = pd.read_parquet("../dataset_files/backtranslate_best_of_n.parquet")

/var/folders/h0/6v9c2xns5lbc32tsmk83rh000000gn/T/ipykernel_5987/3278061739.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = dataset_pd.groupby('instruction_id').apply(pick_completions).reset_index()


In [78]:
result

,instruction_id,chosen,rejected,score_chosen,score_rejected,model_chosen,model_rejected
0,0,"Years after a botched smuggling run, the freig...",In the Star Wars short story from the *Tales f...,262.0,255.0,gemini-2.0-flash,gpt-4o
1,1,"In the heart of the chaotic prison, where shad...",The suppressed memories crashed against Elias’...,272.0,245.0,gpt-4o-mini,gemini-2.0-flash
2,2,"The obsidian portal shrieked shut, collapsing ...","In the heart of the shattered realm, where the...",268.0,257.0,gemini-2.0-flash,gpt-4o-mini
3,3,The air was thick with the stench of iron and ...,"The chamber reeked of iron and despair, the ai...",264.0,245.0,gpt-4o-mini,gpt-4o
4,4,"**A**: Welcome to the lab, B! I'm really excit...","**Setting:** A sterile research facility, fill...",257.0,238.0,gpt-4o,gpt-4o-mini
...,...,...,...,...,...,...,...
10328,10345,The shimmering distortion slammed into Teg’s s...,Teg’s heart raced as he stumbled into the cham...,249.0,245.0,gemini-2.0-flash,gpt-4o-mini
10329,10346,The Director approached the imposing entrance ...,The Director approached the entrance to the se...,256.0,245.0,gpt-4o-mini,gpt-4o
10330,10347,"The EMP screamed through the data center, plun...",The dimly lit corridor erupted into chaos as t...,256.0,243.0,gemini-2.0-flash,gpt-4o
10331,10348,Inside the cramped confines of the battered lu...,Brian sat in the driver's seat of the lunar va...,259.0,236.0,gpt-4o-mini,gpt-4o


In [ ]:
wrapper.init_trainer()

In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
wrapper.train()